In [1]:
import pandas as pd

In [6]:
from yt_dlp import YoutubeDL

In [20]:
urls = ['https://www.youtube.com/watch?v=pqFD6psRDm8']
ydl_opts = {
    'format': 'm4a/bestaudio/best',
    # ℹ️ See help(yt_dlp.postprocessor) for a list of available Postprocessors and their arguments
    'postprocessors': [{  # Extract audio using ffmpeg
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'mp3',
    }]
}

In [21]:
with YoutubeDL(ydl_opts) as ydl:
    ydl.download(urls)

[youtube] Extracting URL: https://www.youtube.com/watch?v=pqFD6psRDm8
[youtube] pqFD6psRDm8: Downloading webpage
[youtube] pqFD6psRDm8: Downloading ios player API JSON
[youtube] pqFD6psRDm8: Downloading m3u8 information
[info] pqFD6psRDm8: Downloading 1 format(s): 140
[download] 不要瞎掰好嗎 [pqFD6psRDm8].m4a has already been downloaded
[download] 100% of  145.17KiB
[ExtractAudio] Destination: 不要瞎掰好嗎 [pqFD6psRDm8].mp3
Deleting original file 不要瞎掰好嗎 [pqFD6psRDm8].m4a (pass -k to keep)
